## Library imports

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

## Constants

In [2]:
TRAIN_NUM_TO_KEEP = 7000
VAL_NUM_TO_KEEP = 2000
DATA_ROOT_DIR = '/home/surya/Downloads/cam2bev-data-master/1_FRLR'
TRAIN_DIR = os.path.join(DATA_ROOT_DIR, *['train'])
VAL_DIR = os.path.join(DATA_ROOT_DIR, *['val'])

## Helper functions

In [3]:
def getFilesInDir(directory):
    return sorted(os.listdir(directory))

def fileNamesWithoutExtension(files):
    return [x.split('.')[-2] for x in files]

def checkFoldersContainSameFiles(folders):
    assert len(folders) > 0
    refFiles = getFilesInDir(folders[0])
    refFilesWithoutExt = set(fileNamesWithoutExtension(refFiles))
    numRefFiles = len(refFiles)
    
    filesMatch = True
    for folder in folders[1:]:
        files = getFilesInDir(folder)
        if (len(files) == numRefFiles):
            filesWithoutExt = set(fileNamesWithoutExtension(files))
            if(len(filesWithoutExt - refFilesWithoutExt) == 0):
                continue
            else:
                filesMatch = False
                print(f"{folder} file names mismatch")
                break
        else:
            filesMatch = False
            print(f"{folder} contains {len(files)} files, while numRefFiles = {numRefFiles}")
            break
            
    return filesMatch

def getRandomIndices(size, numToKeep):
    indices = np.random.choice(size, size=numToKeep, replace=False)
    return indices

def filterListByIndices(data, indices):
    return [data[i] for i in indices]

def deleteFile(filePath):
    if os.path.exists(filePath):
        os.remove(filePath)

def reduceDataset(inputDir, numberToKeep):
    DATASET_FOLDERS = [os.path.join(inputDir,x) for x in os.listdir(inputDir) \
                            if os.path.isdir(os.path.join(inputDir,x))]

    refFiles = getFilesInDir(DATASET_FOLDERS[0])
    numRefFiles = len(refFiles)

    # randomly choose indices to delete
    indicesToDelete = getRandomIndices(numRefFiles, numRefFiles - numberToKeep)
    filesToBeDeteled = filterListByIndices(refFiles, indicesToDelete)

    # delete extra files
    for folder in DATASET_FOLDERS:
        print(folder)
        for file in tqdm(filesToBeDeteled):
            absFilePath = os.path.join(folder, file)
            deleteFile(absFilePath)

    # check folders for files
    checkFoldersContainSameFiles(DATASET_FOLDERS)        

In [4]:
reduceDataset(TRAIN_DIR, numberToKeep=TRAIN_NUM_TO_KEEP)
reduceDataset(VAL_DIR, numberToKeep=VAL_NUM_TO_KEEP)

/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/left


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 38884.63it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/right


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 41234.63it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/front


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1731/1731 [00:00<00:00, 42927.32it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/bev+occlusion


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1731/1731 [00:00<00:00, 42196.32it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/homography


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1731/1731 [00:00<00:00, 39895.05it/s]


In [5]:
def resizeDataset(inputDir, newWidth, newHeight):
    DATASET_FOLDERS = [os.path.join(inputDir,x) for x in os.listdir(inputDir) \
                            if os.path.isdir(os.path.join(inputDir,x))]

    for folder in DATASET_FOLDERS:
        print(folder)
        filesInDir = getFilesInDir(folder)
        for file in tqdm(filesInDir):
            absFilePath = os.path.join(folder, file)
            image = cv2.imread(absFilePath)
            image = cv2.resize(image, (newWidth, newHeight), interpolation=cv2.INTER_CUBIC)
            cv2.imwrite(absFilePath, image)

In [10]:
resizeDataset(TRAIN_DIR, newWidth=512, newHeight=256)

/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/left


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:56<00:00, 124.51it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/right


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:55<00:00, 125.61it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/front


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:57<00:00, 122.31it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/bev+occlusion


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:57<00:00, 122.65it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/train/homography


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:58<00:00, 120.48it/s]


In [11]:
resizeDataset(VAL_DIR, newWidth=512, newHeight=256)

/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/left


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:16<00:00, 121.64it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/right


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:16<00:00, 123.58it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/front


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:16<00:00, 122.33it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/bev+occlusion


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:16<00:00, 122.96it/s]


/home/surya/Downloads/cam2bev-data-master/1_FRLR/val/homography


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:16<00:00, 120.03it/s]
